# 深度学习激活函数详解

## 激活函数的作用

激活函数为神经网络引入非线性特性。没有激活函数，无论网络多深，都等价于单层线性变换。

## 激活函数演进历史

```
sigmoid/tanh → ReLU → Leaky ReLU → PReLU → ELU → SELU → Swish/GELU
```

## 主要问题与解决方案

| 问题 | 影响 | 解决方案 |
|------|------|----------|
| 梯度饱和 | sigmoid/tanh 在极值区梯度趋近于0 | 使用 ReLU 系列 |
| Dead ReLU | 负输入永久置零，神经元"死亡" | Leaky ReLU, PReLU, ELU |
| 输出不对称 | ReLU 输出均值不为0，影响下一层 | ELU, SELU |
| 计算效率 | 某些激活函数计算代价高 | ReLU 简单高效 |

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt

# 设置随机种子
tf.random.set_seed(42)
np.random.seed(42)

print(f"TensorFlow 版本: {tf.__version__}")

## 1. ReLU (Rectified Linear Unit)

**公式**: f(x) = max(0, x)

**优点**:
- 计算简单高效
- 缓解梯度消失问题
- 稀疏激活（约50%神经元输出为0）

**缺点**:
- Dead ReLU 问题：负输入恒为0，梯度为0，神经元可能永久"死亡"

In [ ]:
# ReLU 激活函数可视化
x = np.linspace(-5, 5, 200)

# ReLU: max(0, x)
relu = np.maximum(0, x)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 函数图像
axes[0].plot(x, relu, 'b-', linewidth=2, label='ReLU')
axes[0].axhline(y=0, color='k', linestyle='-', linewidth=0.5)
axes[0].axvline(x=0, color='k', linestyle='-', linewidth=0.5)
axes[0].set_xlabel('x')
axes[0].set_ylabel('f(x)')
axes[0].set_title('ReLU: f(x) = max(0, x)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 导数图像
relu_grad = np.where(x > 0, 1, 0)
axes[1].plot(x, relu_grad, 'r-', linewidth=2, label="ReLU'")
axes[1].axhline(y=0, color='k', linestyle='-', linewidth=0.5)
axes[1].axvline(x=0, color='k', linestyle='-', linewidth=0.5)
axes[1].set_xlabel('x')
axes[1].set_ylabel("f'(x)")
axes[1].set_title('ReLU 导数')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Leaky ReLU

**公式**: f(x) = x if x > 0 else αx (通常 α = 0.01 或 0.3)

**优点**: 解决 Dead ReLU 问题，负区域有小梯度

**缺点**: α 是超参数，需要调优

In [ ]:
# Keras 中使用 Leaky ReLU
# 方式1: 作为独立层使用
model_leaky = keras.Sequential([
    layers.Dense(64, kernel_initializer='he_normal'),
    layers.LeakyReLU(negative_slope=0.3),  # 负斜率为 0.3
    layers.Dense(10, activation='softmax')
])

# 可视化不同 alpha 值的 Leaky ReLU
alphas = [0.01, 0.1, 0.3]
colors = ['blue', 'green', 'red']

plt.figure(figsize=(10, 4))

for alpha, color in zip(alphas, colors):
    leaky_relu = np.where(x > 0, x, alpha * x)
    plt.plot(x, leaky_relu, color=color, linewidth=2, label=f'α = {alpha}')

plt.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
plt.axvline(x=0, color='k', linestyle='-', linewidth=0.5)
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('Leaky ReLU with different α values')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("Leaky ReLU 模型构建成功")

## 3. PReLU (Parametric ReLU)

**公式**: f(x) = x if x > 0 else αx，其中 α 是可学习参数

**优点**: α 通过反向传播自动学习，无需手动调优

**缺点**: 增加模型参数量

In [ ]:
# Keras 中使用 PReLU
model_prelu = keras.Sequential([
    layers.Dense(64, kernel_initializer='he_normal'),
    layers.PReLU(),  # α 是可学习参数
    layers.Dense(32, kernel_initializer='he_normal'),
    layers.PReLU(),
    layers.Dense(10, activation='softmax')
])

# 构建模型以查看参数
model_prelu.build(input_shape=(None, 784))
model_prelu.summary()

print("\nPReLU 模型构建成功")
print("注意: PReLU 层有可学习参数 alpha")

## 4. ELU (Exponential Linear Unit)

**公式**: f(x) = x if x > 0 else α(e^x - 1)

**优点**:
- 输出均值接近0（自归一化特性）
- 负区域平滑过渡，梯度不会突变
- 缓解 Dead ReLU 问题

**缺点**: 计算代价比 ReLU 高（涉及指数运算）

In [ ]:
# ELU 可视化
alpha = 1.0
elu = np.where(x > 0, x, alpha * (np.exp(x) - 1))

plt.figure(figsize=(10, 4))

# 对比 ReLU 和 ELU
plt.plot(x, relu, 'b--', linewidth=2, label='ReLU', alpha=0.7)
plt.plot(x, elu, 'r-', linewidth=2, label=f'ELU (α={alpha})')

plt.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
plt.axvline(x=0, color='k', linestyle='-', linewidth=0.5)
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('ELU vs ReLU')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(-2, 5)
plt.show()

# Keras 中使用 ELU
model_elu = keras.Sequential([
    layers.Dense(64, activation='elu', kernel_initializer='he_normal'),
    layers.Dense(32, activation='elu', kernel_initializer='he_normal'),
    layers.Dense(10, activation='softmax')
])

print("ELU 模型构建成功")

## 5. SELU (Scaled ELU)

**公式**: f(x) = λ × (x if x > 0 else α(e^x - 1))

其中 λ ≈ 1.0507, α ≈ 1.6733 是精心选择的常数。

**优点**: 自归一化特性 - 在特定条件下，网络输出自动保持均值为0、方差为1

**使用条件**:
- 必须使用 `lecun_normal` 初始化
- 必须使用 `AlphaDropout`（而非普通 Dropout）
- 网络必须是全连接的（Sequential Dense layers）

In [ ]:
# SELU 自归一化网络示例
model_selu = keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    # SELU 必须配合 lecun_normal 初始化
    layers.Dense(256, activation='selu', kernel_initializer='lecun_normal'),
    layers.AlphaDropout(0.1),  # SELU 专用 Dropout
    layers.Dense(128, activation='selu', kernel_initializer='lecun_normal'),
    layers.AlphaDropout(0.1),
    layers.Dense(64, activation='selu', kernel_initializer='lecun_normal'),
    layers.Dense(10, activation='softmax')
])

model_selu.summary()
print("\nSELU 自归一化网络构建成功")

## 6. 所有激活函数对比可视化

In [ ]:
# 计算各激活函数
x = np.linspace(-4, 4, 200)

activations = {
    'ReLU': np.maximum(0, x),
    'Leaky ReLU (α=0.1)': np.where(x > 0, x, 0.1 * x),
    'ELU (α=1)': np.where(x > 0, x, 1.0 * (np.exp(x) - 1)),
    'SELU': 1.0507 * np.where(x > 0, x, 1.6733 * (np.exp(x) - 1)),
    'tanh': np.tanh(x),
    'sigmoid': 1 / (1 + np.exp(-x))
}

fig, axes = plt.subplots(2, 3, figsize=(14, 8))
axes = axes.flatten()

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']

for idx, (name, values) in enumerate(activations.items()):
    axes[idx].plot(x, values, color=colors[idx], linewidth=2)
    axes[idx].axhline(y=0, color='k', linestyle='-', linewidth=0.5)
    axes[idx].axvline(x=0, color='k', linestyle='-', linewidth=0.5)
    axes[idx].set_title(name, fontsize=12)
    axes[idx].set_xlabel('x')
    axes[idx].set_ylabel('f(x)')
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_xlim(-4, 4)
    axes[idx].set_ylim(-2, 4)

plt.suptitle('深度学习常用激活函数对比', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('activation_functions_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. 实际应用：构建完整模型

In [ ]:
def build_model_with_activation(activation_name, use_bn=False):
    """
    构建使用指定激活函数的模型
    
    Parameters:
    -----------
    activation_name : str
        激活函数名称: 'relu', 'leaky_relu', 'prelu', 'elu', 'selu'
    use_bn : bool
        是否使用批量归一化
    
    Returns:
    --------
    keras.Model
        编译好的模型
    """
    # 根据激活函数选择初始化方法
    if activation_name == 'selu':
        initializer = 'lecun_normal'
    else:
        initializer = 'he_normal'
    
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    
    for units in [256, 128, 64]:
        if activation_name == 'leaky_relu':
            model.add(layers.Dense(units, kernel_initializer=initializer))
            model.add(layers.LeakyReLU(negative_slope=0.2))
        elif activation_name == 'prelu':
            model.add(layers.Dense(units, kernel_initializer=initializer))
            model.add(layers.PReLU())
        else:
            model.add(layers.Dense(units, activation=activation_name, 
                                  kernel_initializer=initializer))
        
        if use_bn and activation_name != 'selu':
            model.add(layers.BatchNormalization())
        elif activation_name == 'selu':
            model.add(layers.AlphaDropout(0.1))
    
    model.add(layers.Dense(10, activation='softmax'))
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# 测试各种激活函数的模型构建
activation_list = ['relu', 'leaky_relu', 'prelu', 'elu', 'selu']

for act in activation_list:
    model = build_model_with_activation(act)
    param_count = model.count_params()
    print(f"{act.upper():12s} - 参数量: {param_count:,}")

## 8. 激活函数选择指南

### 推荐策略

| 场景 | 推荐激活函数 | 配套设置 |
|------|--------------|----------|
| 默认选择 | ReLU | He 初始化 |
| 担心 Dead ReLU | Leaky ReLU / ELU | He 初始化 |
| 自归一化网络 | SELU | LeCun 初始化 + AlphaDropout |
| 二分类输出层 | sigmoid | - |
| 多分类输出层 | softmax | - |
| RNN 隐藏层 | tanh | - |

### 性能对比（一般规律）

- **训练速度**: ReLU > Leaky ReLU > ELU > SELU
- **模型性能**: 通常差异不大，ELU/SELU 在深层网络中略有优势
- **稳定性**: SELU（满足条件时）> ELU > Leaky ReLU > ReLU

In [ ]:
# 验证代码正确性
print("激活函数模块测试完成")
print("\n关键要点:")
print("1. ReLU 是默认首选，简单高效")
print("2. Leaky ReLU/PReLU/ELU 解决 Dead ReLU 问题")
print("3. SELU 实现自归一化，需要特定配置")
print("4. 输出层使用 sigmoid（二分类）或 softmax（多分类）")